In [1]:
from simple_cysh import * # includes pandas as pd

In [24]:
cysh = init_cysh()

In [25]:
# determine fields used within object
single_tut_sess = cysh.Intervention_Session_Result__c.get('a0f1a00000CyUX5AAN')
single_tut_sess = pd.DataFrame(list(single_tut_sess.items()))
list(single_tut_sess.loc[~single_tut_sess[1].isnull()][0])

['attributes',
 'Id',
 'IsDeleted',
 'Name',
 'CreatedDate',
 'CreatedById',
 'LastModifiedDate',
 'LastModifiedById',
 'SystemModstamp',
 'LastViewedDate',
 'LastReferencedDate',
 'Intervention_Session__c',
 'Student_Section__c',
 'Amount_of_Time__c',
 'Intervention_Session_Date__c',
 'Program_Name__c',
 'Program__c',
 'Related_Student_s_Name__c',
 'Section_Name__c',
 'Session_DOW__c',
 'Student_Reference_ID__c',
 'Primary_Skill__c']

In [ ]:
single_staff = cysh.Staff__c.get('a1L1a000003JsZWEA0')
single_staff = pd.DataFrame(list(single_staff.items()))
list(single_staff.loc[~single_staff[1].isnull()][0])

In [37]:
single_student_section = cysh.Student_Section__c.get('a1T1a000001sEN2EAM')
single_student_section = pd.DataFrame(list(single_student_section.items()))
sorted(list(single_student_section.loc[~single_student_section[1].isnull()][0]))

['Active__c',
 'Amount_of_Time__c',
 'Archived__c',
 'Average_Section_Grade__c',
 'Count_Section_Grades__c',
 'CreatedById',
 'CreatedDate',
 'Deactivated__c',
 'Dosage_to_Date__c',
 'Dosage_to_Date_hrs__c',
 'Enrollment_Start_Date__c',
 'Failing__c',
 'Id',
 'IsDeleted',
 'Is_Section__c',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDate',
 'LastViewedDate',
 'Local_Student_ID__c',
 'Location__c',
 'Name',
 'Reference_ID__c',
 'School_Reference_Id__c',
 'School_Year_Name__c',
 'School__c',
 'Section_Attendance_Flag__c',
 'Section_Name__c',
 'Section_Period__c',
 'Section_RT__c',
 'Section_ReportingPeriod_StartDate__c',
 'Section__c',
 'Section_time__c',
 'Setup_Year_Name__c',
 'Student_Entry_Date__c',
 'Student_First_Name__c',
 'Student_Grade__c',
 'Student_Id__c',
 'Student_Name__c',
 'Student__c',
 'Student_school_year__c',
 'SystemModstamp',
 'Total_Grade_Value__c',
 'attributes']

In [ ]:
single_section = cysh.Section__c.get('a1A1a000004QJ9kEAG')
single_section = pd.DataFrame(list(single_section.items()))
sorted(list(single_section.loc[~single_section[1].isnull()][0]))

In [ ]:
# Fill in details on each section, enure no duplicates (one staff per section)
section_df = get_cysh_df('Section__c', ['Id', 'Name', 'School__c', 'Program__c', 'In_After_School__c', 'Intervention_Primary_Staff__c'], rename_id=True)
section_df.rename(columns={'Intervention_Primary_Staff__c':'Staff__c'}, inplace=True)
program_df = get_cysh_df('Program__c', ['Id', 'Name'], rename_id=True, rename_name=True)
staff_df = get_cysh_df('Staff__c', ['Id', 'Name', 'Role__c'], rename_id=True, rename_name=True)

In [ ]:
# add program tpye to section
df = section_df.merge(program_df, how='left', on='Program__c')
# add staff 'Name' and 'Role__c' to staff_section
df = df.merge(staff_df, how='left', on='Staff__c')
# filter
df = df.loc[~df['Program__c_Name'].isnull()]
df = df.loc[df['Program__c_Name'].str.contains('Tutoring')
            & (df['In_After_School__c']!='Extended Learning')]

In [19]:
student_section_df = get_cysh_df('Student_Section__c', ['Id', 'Section__c', 'Student__c'], rename_id=True)
ISR_df = get_cysh_df('Intervention_Session_Result__c', ['CreatedDate', 'Intervention_Session_Date__c',
                                                        'Amount_of_Time__c', 'Primary_Skill__c',
                                                        'Student_Section__c', 'Intervention_Session__c',
                                                        'Section_Name__c', 'Program__c', 'Program_Name__c'])

In [35]:
ISR_df = ISR_df.merge(student_section_df, how='left', on='Student_Section__c')
ISR_df['student_program'] = ISR_df['Student__c'] + '_' + ISR_df['Program__c']
ISR_df['Intervention_Session_Date__c'] = pd.to_datetime(ISR_df['Intervention_Session_Date__c'])
ISR_df = ISR_df.loc[ISR_df['Intervention_Session_Date__c'] < '2017-12-11']
ISR_df = ISR_df.join(ISR_df.groupby('student_program')['Amount_of_Time__c'].sum(), how='left', on='student_program', rsuffix='_YTD')

In [59]:
tot_df = ISR_df.groupby(['Student__c', 'Program__c', 'Program_Name__c'])['Amount_of_Time__c'].sum().reset_index()

In [60]:
tot_df.to_csv('privatedata/time_on_task_2017-12-11.csv', index=False)